# Chemical Kinetics (review on Linear Equations)

In this tutorial, we will review Linear Equations and discuss three different scenarios,
1. Normal (same number of columns and rows)
2. Overdetermined (more rows than columns)
3. Undermined (more columns than rows)
  
This code will also show us how to solve all these different cases. <br>

This tutorial can be deployed in <a target="_blank" href="https://colab.research.google.com/github/ChemAI-Lab/Math4Chem/blob/main/website/Lecture_Notes/Coding/chem_kinetics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Linear Equations

A set of linear equations has the following structure,
$$
\mathbf{A} \mathbf{x} = \mathbf{b}
$$
where $\mathbf{A}$ a matrix with $n$-rows and $m$-columns, the vector $\mathbf{x}$ must have $m$-entries, and $\mathbf{b}$ is a vector with $n$-entries. <br>
We saw this in linear regression where $\mathbf{x}$ was the vector of the parameters of the model, for example, $\mathbf{x}^\top = [a, b]$, and each row of $\mathbf{A}$ was a single data point.

# Square matrices

A square matrix is a matrix that has an equal number of rows and columns,
$$
\mathbf{A} = \begin{bmatrix} a_{11} & a_{12} & \cdots & a_{1n} \\\\ a_{21} & a_{22} & \cdots & a_{2n} \\\\ \vdots & \vdots & \ddots & \vdots \\\\ a_{n1} & a_{n2} & \cdots & a_{nn} \end{bmatrix}
$$
We can invert a square matrix, $\mathbf{A}^{-1}\mathbf{A} = \mathbf{I}$, where $\mathbf{I}$ is the identity matrix. <br>
* $\mathbf{A}$ is invertible, if its determinant is not zero.

### Solving a system of linear equations using the inverse of a matrix.
$$
\mathbf{A} \mathbf{x} = \mathbf{b}\\
\underbrace{\mathbf{A}^{-1}\mathbf{A}}_{\mathbf{I}} \mathbf{x} = \mathbf{A}^{-1}\mathbf{b}\\
\mathbf{x} = \mathbf{A}^{-1} \mathbf{b}
$$

If we can invert $\mathbf{A}$, we just need to multiply its inverse with $\mathbf{b}$, to find $\mathbf{x}$.

## Determining the Rate Law from Experimental Data

To illustrate the solution of Linear equations when $\mathbf{A}$ is invertible, we are going to use the following example, [link](https://flexbooks.ck12.org/cbook/ck-12-chemistry-flexbook-2.0/section/18.10/primary/lesson/determining-the-rate-law-from-experimental-data-chem/).<br>

Let's consider the following reaction,
$$
2NO(g) + 2H_2(g) \to N_2(g) + 2H_2O(g)
$$

Then we collected the following measured the concentrations of this reactions and it's rates, 

| Experiment | [NO] | [H<sub>2</sub>] | Initial Rate [M/s]
| :---: | :---: | :---: | :---: |
| 1 | 0.005 | 0.002 | -11.29 |
| 2 | 0.01 | 0.002 | -9.90 |
| 3 | 0.01 | 0.004 | -9.21 |


We can write the rates of the reactions in terms of the concentrations of the reactants using the following rate law,
$$
r = k [NO]^{\alpha} [H_2]^{\beta},
$$
This equation does not look linear, but using the properties of logarithms we can rewrite it as,

$$
\ln r = \ln k + \alpha \ln [NO] + \beta \ln [H_2]
$$
We can solve this system of linear equations to find the values of $\mathbf{x}^\top =  [\ln k, \alpha, \beta]$.
Each of the rows of $\mathbf{A}$ will have the following representation, $\mathbf{a}_i^\top = [1, \ln [NO], \ln [H_2]]$, and $\mathbf{b} = \ln (\text{Initial Rates})$.


In [ ]:
A = np.array(
    #code here
)
A = np.log(A)
# why the np.ones(3,)?
A = # code here

b = np.array([#code here])
b = np.log(b)

# solve for x
# code here

ln_k = x[0]
alpha = x[1]
beta = x[2]
print(f"ln(k) = {ln_k:.4f}", f"k = {np.exp(ln_k):.4f}")
print(f"alpha = {alpha:.2f}")
print(f"beta = {beta:.2f}")

# you can compare your results with
x_numpy = np.linalg.solve(A, b)

ln_k = x_numpy[0]
alpha = x_numpy[1]
beta = x_numpy[2]
print(f"ln(k) = {ln_k:.4f}", f"k = {np.exp(ln_k):.4f}")
print(f"alpha = {alpha:.2f}")
print(f"beta = {beta:.2f}")

# Types of Non-Square Matrix solutions (Overdetermined)

Linear equations as you know could also describe problems where have either more points (rows) than features (columns), ir vice versa more features (columns) than points (rows).<br>
The case where the number of points is bigger than the number of columns is known as **Overdetermined**. 
Because $\mathbf{A}$ is not a square matrix, 
$$
\mathbf{A} = \begin{bmatrix} a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\\\ a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\\\ \vdots & \vdots & \ddots & \vdots \\ \\ a_{n-1,1} & a_{n-1, 2} & \cdots & a_{n-1, m} \\ a_{n,1} & a_{n,2} & \cdots & a_{n,m} \end{bmatrix},
$$
where $n > m$.<br>
We can approximate the inverse of these matrix using Singular Value Decomposition (SVD) [link](https://sthalles.github.io/svd-for-regression/).

## Singular Value Decomposition
The SVD of a matrix $\mathbf{A}$ is given by
$$
\mathbf{A} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T
$$
where $\mathbf{U}$ and $\mathbf{V}^T$ are orthogonal matrices,
$$
\mathbf{U}^T \mathbf{U} = \mathbf{I} \\
\mathbf{V}^T \mathbf{V} = \mathbf{I},
$$
and $\mathbf{\Sigma}$ is a diagonal matrix with non-negative real numbers on the diagonal.


Due to the shortness of the course we will not properly review SVDs, but is one of the main building blocks in scientific computing. <br>


## Linear Equations using SVD
The idea if SVD is to approximate the inverse of $\mathbf{A}$. 
$$
\mathbf{A} \mathbf{x} = \mathbf{b} \\
\underbrace{\left(\mathbf{U}\mathbf{\Sigma}\mathbf{V}^\top \right)}_{\mathbf{A}}\mathbf{x} =\mathbf{b} \\
\left(\mathbf{U}\mathbf{\Sigma}\mathbf{V}^\top \right)^{-1} \left(\mathbf{U}\mathbf{\Sigma}\mathbf{V}^\top \right)\mathbf{x} = \left(\mathbf{U}\mathbf{\Sigma}\mathbf{V}^\top \right)^{-1} \mathbf{b} \\
\mathbf{V}\mathbf{\Sigma}^{-1}\mathbf{U}^\top \mathbf{U}\mathbf{\Sigma}\mathbf{V}^\top \mathbf{x} = \left(\mathbf{V}\mathbf{\Sigma}^{-1}\mathbf{U}^\top \right) \mathbf{b},
$$
whe can cancel some terms, using the orthogonality property of $\mathbf{U}$ and $\mathbf{V}$, giving us, 
$$
\mathbf{x} = \mathbf{V}\mathbf{\Sigma}^{-1}\mathbf{U}^\top \mathbf{b} \\
\mathbf{x} = \mathbf{A}^{+} \mathbf{b},
$$
where $\mathbf{A}^{+}$ is known as the *pseudo* inverse of $\mathbf{A}$; $\mathbf{A}^{+} \mathbf{A} \approx \mathbf{I}$.


Extra:<br>
* $\left(\mathbf{U}\mathbf{\Sigma}\mathbf{V}^\top \right)^{-1} = \left(\mathbf{V}\mathbf{\Sigma}^{-1}\mathbf{U}^\top \right)$ is explained in Eq. 223 of [The Matrix Cookbook](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf)


You can get the SVD of any matrix in Numpy using the following code, 

```python
import numpy as np
X = np.random.rand(10, 3) # 10 rows, 3 columns
# Perform SVD on X
U, S, Vt = np.linalg.svd(X, full_matrices=False)
```

In [ ]:
import numpy as np
A = np.random.rand(10, 3)  # 10 rows, 3 columns
print('Matrix A')
print(A)
print('----------------------------------------------------------------')
# Perform SVD on X
U, S, Vt = np.linalg.svd(A, full_matrices=False)
V = Vt.T
print('U: ',U.shape, 'S: ', S.shape, 'V:', V.shape)
print(U.T@U)
print(Vt@V)
print(S)
print('----------------------------------------------------------------')
S_inv = np.diag(1 / S)
A_pinv = Vt.T @ S_inv @ U.T
print('Psuedoinverse of A')
print(A_pinv.shape)
print(A_pinv)
print('----------------------------------------------------------------')
# reconstruct A = U Sigma V^T
A_rec = U @ np.diag(S) @ Vt
print('Reconstructed Matrix A')
print(A_rec)
print('----------------------------------------------------------------')
print('A+ A = I')
print(A_pinv@A)
print('----------------------------------------------------------------')

In [ ]:
# how many components de we need to reconstruct A
for i in range(3): 
    A_a = U[:, :i] @ np.diag(S[:i]) @ Vt[:i,:]
    error = np.linalg.norm(A_a - A)
    print(f"Error in reconstructed matrix for {i+1} components: {error:.4e}")

Let's consider the following data, where we have more experiments than partial orders and rate constant. Example from [link](https://chem.libretexts.org/Bookshelves/General_Chemistry/Chemistry_1e_(OpenSTAX)/12%3A_Kinetics/12.04%3A_Rate_Laws). <br>

The reaction we are studying is, 
$$
NO(g) + O_3(g) \to NO_2(g) + O_2(g)
$$

Then we collected the following measured the concentrations of this reactions and it's rates, 

| Experiment | [NO] | [O<sub>4</sub>] | Initial Rate [M/s]
| :---: | :---: | :---: | :---: |
1 |	1.00 × 10−6 |	3.00 × 10−6 |	6.60 × 10−5
2 |	1.00 × 10−6 |	6.00 × 10−6 |	1.32 × 10−4
3 |	1.00 × 10−6 |	9.00 × 10−6 |	1.98 × 10−4
4 |	2.00 × 10−6 |	9.00 × 10−6 |	3.96 × 10−4
5 |	3.00 × 10−6 |	9.00 × 10−6 |	5.94 × 10−4


The rate law for this reaction has the following form,
$$
r = k [NO]^{\alpha}[O_3]^{\beta}.
$$

*We could choose only 3 of those experiments randomly and solve this problem using the square matrix approach.<br> 
I leave you this as exercise/homework, you can also average these numbers over different random experiments selected multiple times. <br>


Here, we will choose the SVD approach to find $\ln k$, $\alpha$, and $\beta$.<br>
Again, we can transform the rate law into a linear system of equations using the logarithm,
$$
\ln r = \ln k +  \alpha \ln [NO] + \beta\ln [O_3].
$$


In [ ]:
# Linear Equations using SVD
A = np.array([[1E-6,3E-6],
             [1E-6,6E-6],
             [1E-6,9E-6],
             [2E-6,9E-6],
             [3E-6,9E-6]])
A = np.log(A)
A = np.column_stack((np.ones(A.shape[0],), A))  # why the np.ones(3,)?

b = np.array([6.6E-5,1.32E-4,1.98E-4,3.96E-4,5.94E-4])
b = np.log(b)

# Step 1: Perform SVD on X


# Step 2: Compute the pseudo-inverse of the singular values


# Step 3: Calculate the pseudo-inverse of X using U, S_inv, and Vt


# Step 4: Compute the weight vector (regression coefficients) w


ln_k = x[0]
alpha = x[1]
beta = x[2]
print(f"ln(k) = {ln_k:.4f}", f"k = {np.exp(ln_k):.4f}")
print(f"alpha = {alpha:.2f}")
print(f"beta = {beta:.2f}")


# Extra

## Steady State Solution using gradient descent
Let's consider the following reaction,
$$
2NO + Br_2 \to 2NOBr
$$
with the forward and reverse equilibrium constants ($k_f$ and $k_r$).

We can derive three different rate laws for each species,
$$
\frac{d [NO]}{dt} = 2k_b[NOBr]^2 - 2k_f[Br_2][NO]^2\\

\frac{d [Br_2]}{dt} = k_b[NOBr]^2 - k_f[Br_2][NO]^2\\

\frac{d [NOBr]}{dt} = 2k_f[Br_2][NO]^2 - 2k_b[NOBr]^2
$$

The goal is to find the values of $[NO], [Br_2]$ and $[NOBr]$,  where $\frac{d [NO]}{dt} = \frac{d [Br_2]}{dt} = \frac{d [NOBr]}{dt} = 0$.<br>


For these conditions we can use gradient descent to find these steady state concentrations.

## Time-dependent Ordinary differential equations.

For the previous reaction, we see that this is a time dependent process, and the steady-state solution only tells us the concentrations at equilibrium but not the time it takes to reach the equilibrium of the reaction.<br>
To solve this problem, we can use the method of numerical integration.<br>
For example, we can use the Euler method, which we will cover in the next class, or other method of numerical integration for ODEs. <br>

The main idea of an ODE is to solve the differential equations in time, giving us, for this case, the concentration of each species at different times.<br>
For this, we will use the Scipy library, which contains many numerical integrators for ODEs./


In [ ]:
from scipy.integrate import odeint
def equations_of_motion(x, t, kf, kb):
    rf = kf * x[0]**2 * x[1]
    rb = kb * x[2]**2
    return [2*(rb - rf), rb - rf, 2*(rf - rb)]

t_grid = np.linspace(0,20,100)
k_vals = 0.42, 0.17 #kf, kb
y0 = np.array([1, 1, 0]) # initial concentrations
y_tgrid = odeint(equations_of_motion, y0, t_grid, k_vals)  # EXERCISE: rhs, y0, tout, k_vals
print(y_tgrid)

print('Steady State')
xss_odeint = y_tgrid[-1, :]
print(xss_odeint)

In [ ]:
plt.plot(t_grid, y_tgrid)
_ = plt.legend(['NO', 'Br$_2$', 'NOBr'])

# Gradient Descent for the Steady State
For these conditions we can use gradient descent to find these steady state concentrations.

In [ ]:
def equations_of_motion(x, kf, kb):
    rf = kf * x[0]**2 * x[1]
    rb = kb * x[2]**2
    return [2*(rb - rf), rb - rf, 2*(rf - rb)]


def gradient_of_equations_of_motion(x, kf, kb):
    dg1_dx1 = -4*kf*x[1]*x[0]
    dg1_dx2 = -2*kf*x[0]**2
    dg1_dx3 = 4*kb*x[2]

    dg2_dx1 = -2*kf*x[1]*x[0]
    dg2_dx2 = -kf*x[0]**2
    dg2_dx3 = 2*kb*x[2]

    dg3_dx1 = 4*kf*x[1]*x[0]
    dg3_dx2 = 2*kf*x[0]**2
    dg3_dx3 = -4*kb*x[2]

    jac = np.array([[dg1_dx1, dg1_dx2, dg1_dx3],
                    [dg2_dx1, dg2_dx2, dg2_dx3],
                    [dg3_dx1, dg3_dx2, dg3_dx3]])
    return jac


def error_function(x, kf, kb):
    g = equations_of_motion(x, kf, kb)
    return 0.5 * np.dot(g, g)


def gradient_error_function(x, kf, kb):
    g = equations_of_motion(x, kf, kb)
    jac_g = gradient_of_equations_of_motion(x, kf, kb)
    return jac_g.T @ g

In [ ]:
# x0 = np.array([0.45, 0.45, 0.45])
x0 = np.random.uniform(0.,0.01,size=(3,)) + np.array([0.42940398, 0.71470199, 0.57059602])
kf, kb = 0.42, 0.17

eta = 0.02
for i in range(1000):
    dx0 = gradient_error_function(x0, kf, kb)
    e = error_function(x0, kf, kb)
    print(f'Itr {i+1}, e = ', e, 'x = ', x0, '|dx| = ', np.linalg.norm(dx0))
    x0 = x0 - eta * dx0

xss_gd = x0
print('Steady State concentrations')
print(f'[NO] = {x0[0]:.3f}')
print(r'[Br2]' + f' = {x0[1]:.3f}')
print(f'[NOBr] = {x0[2]:.3f}')